In [1]:
# PASSO A PASSO
# PASSO 0 - ENTENDER O DESAFIO DA EMPRESA
# PASSO 1 - IMPORTAR A BASE DE DADOS
# PASSO 2 - PREPARAR A BASE DE DADOS PARA A IA 
# PASSO 3 - CRIAR UM MODELO DE IA (SCORE DE CREDITO)
# PASSO 4 - ESCOLHER O MELHOR MODELO
# PASSO 5 - UTILIZAR A IA PARA REALIZAR AS PREVISÕES


In [2]:
# PASSO 1: Importação das bibliotecas e carregamento dos dados

# Importa as bibliotecas necessárias para análise e modelagem de dados
import pandas
from sklearn.preprocessing import LabelEncoder  # Para codificação de variáveis categóricas
from sklearn.model_selection import train_test_split  # Para divisão dos dados em treino e teste
from sklearn.ensemble import RandomForestClassifier  # Modelo de classificação baseado em árvores
from sklearn.neighbors import KNeighborsClassifier  # Modelo de classificação K-Nearest Neighbors
from sklearn.metrics import accuracy_score  # Métrica para avaliação do modelo

# Carrega os dados do arquivo CSV para um DataFrame
tabela = pandas.read_csv("clientes.csv")

# Exibe as primeiras linhas da tabela carregada
display(tabela)


id_cliente  mes  idade  profissao  salario_anual  num_contas  \
0            3392    1   23.0  cientista       19114.12         3.0   
1            3392    2   23.0  cientista       19114.12         3.0   
2            3392    3   23.0  cientista       19114.12         3.0   
3            3392    4   23.0  cientista       19114.12         3.0   
4            3392    5   23.0  cientista       19114.12         3.0   
...           ...  ...    ...        ...            ...         ...   
99995       37932    4   25.0   mecanico       39628.99         4.0   
99996       37932    5   25.0   mecanico       39628.99         4.0   
99997       37932    6   25.0   mecanico       39628.99         4.0   
99998       37932    7   25.0   mecanico       39628.99         4.0   
99999       37932    8   25.0   mecanico       39628.99         4.0   

       num_cartoes  juros_emprestimo  num_emprestimos  dias_atraso  ...  \
0              4.0               3.0              4.0          3.0  ...   
1              4.0               3.0              4.0          3.0  ...   
2              4.0               3.0              4.0          3.0  ...   
3              4.0               3.0              4.0          5.0  ...   
4              4.0               3.0              4.0          6.0  ...   
...            ...               ...              ...          ...  ...   
99995          6.0               7.0              2.0         23.0  ...   
99996          6.0               7.0              2.0         18.0  ...   
99997          6.0               7.0              2.0         27.0  ...   
99998          6.0               7.0              2.0         20.0  ...   
99999          6.0               7.0              2.0         18.0  ...   

       idade_historico_credito  investimento_mensal  \
0                        265.0            21.465380   
1                        266.0            21.465380   
2                        267.0            21.465380   
3                        268.0            21.465380   
4                        269.0            21.465380   
...                        ...                  ...   
99995                    378.0            24.028477   
99996                    379.0            24.028477   
99997                    380.0            24.028477   
99998                    381.0            24.028477   
99999                    382.0            24.028477   

           comportamento_pagamento  saldo_final_mes  score_credito  \
0      alto_gasto_pagamento_baixos       312.494089           Good   
1       baixo_gasto_pagamento_alto       284.629162           Good   
2      baixo_gasto_pagamento_medio       331.209863           Good   
3      baixo_gasto_pagamento_baixo       223.451310           Good   
4       alto_gasto_pagamento_medio       341.489231           Good   
...                            ...              ...            ...   
99995    alto_gasto_pagamento_alto       479.866228           Poor   
99996   alto_gasto_pagamento_medio       496.651610           Poor   
99997    alto_gasto_pagamento_alto       516.809083           Poor   
99998   baixo_gasto_pagamento_alto       319.164979       Standard   
99999   alto_gasto_pagamento_medio       393.673696           Poor   

       emprestimo_carro  emprestimo_casa emprestimo_pessoal  \
0                     1                1                  1   
1                     1                1                  1   
2                     1                1                  1   
3                     1                1                  1   
4                     1                1                  1   
...                 ...              ...                ...   
99995                 1                0                  0   
99996                 1                0                  0   
99997                 1                0                  0   
99998                 1                0                  0   
99999                 1                0                  

In [3]:
# PASSO 2: Análise para encontrar possíveis erros nos dados

# Exibe informações sobre o DataFrame, como:
# - Número de entradas e colunas
# - Tipos de dados de cada coluna
# - Quantidade de valores não nulos em cada coluna
display(tabela.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 25 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   id_cliente                100000 non-null  int64  
 1   mes                       100000 non-null  int64  
 2   idade                     100000 non-null  float64
 3   profissao                 100000 non-null  object 
 4   salario_anual             100000 non-null  float64
 5   num_contas                100000 non-null  float64
 6   num_cartoes               100000 non-null  float64
 7   juros_emprestimo          100000 non-null  float64
 8   num_emprestimos           100000 non-null  float64
 9   dias_atraso               100000 non-null  float64
 10  num_pagamentos_atrasados  100000 non-null  float64
 11  num_verificacoes_credito  100000 non-null  float64
 12  mix_credito               100000 non-null  object 
 13  divida_total              100000 non-null  fl

None

In [4]:
# PASSO 3: Tratamento de colunas do tipo "object" para conversão em valores numéricos

# Como os modelos de machine learning não trabalham diretamente com textos (colunas do tipo "object"),
# é necessário converter essas colunas em valores numéricos.
# As colunas a serem tratadas são: "profissao", "mix_credito" e "comportamento_pagamento".
# A coluna "score_credito" será mantida sem alterações, pois é a variável alvo (resultado esperado).

# Instancia o codificador LabelEncoder
codificador = LabelEncoder()

# Aplica o LabelEncoder para transformar as colunas categóricas em numéricas
tabela["profissao"] = codificador.fit_transform(tabela["profissao"])
tabela["mix_credito"] = codificador.fit_transform(tabela["mix_credito"])
tabela["comportamento_pagamento"] = codificador.fit_transform(tabela["comportamento_pagamento"])

# Análise após as alterações para verificar se as colunas foram convertidas corretamente
display(tabela.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 25 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   id_cliente                100000 non-null  int64  
 1   mes                       100000 non-null  int64  
 2   idade                     100000 non-null  float64
 3   profissao                 100000 non-null  int32  
 4   salario_anual             100000 non-null  float64
 5   num_contas                100000 non-null  float64
 6   num_cartoes               100000 non-null  float64
 7   juros_emprestimo          100000 non-null  float64
 8   num_emprestimos           100000 non-null  float64
 9   dias_atraso               100000 non-null  float64
 10  num_pagamentos_atrasados  100000 non-null  float64
 11  num_verificacoes_credito  100000 non-null  float64
 12  mix_credito               100000 non-null  int32  
 13  divida_total              100000 non-null  fl

None

In [5]:
# PASSO 3 / PASSO 4: Aprendizado de Máquina

# A coluna "score_credito" é a variável alvo (Y), ou seja, o que queremos prever.
# As demais colunas (exceto "id_cliente" e "score_credito") serão as variáveis preditoras (X),
# utilizadas para realizar a previsão.

# Define a variável alvo (Y) e as variáveis preditoras (X)
y = tabela["score_credito"]
x = tabela.drop(columns=["id_cliente", "score_credito"])

# Divisão dos dados em treino e teste
# O método "train_test_split" divide os dados em 70% para treino e 30% para teste
x_treino, x_test, y_treino, y_test = train_test_split(x, y, test_size=0.3)

# Criação dos modelos de aprendizado de máquina
# RandomForestClassifier: Modelo baseado em árvores de decisão
# KNeighborsClassifier: Modelo baseado em K-Nearest Neighbors
modelo_arvoredecisao = RandomForestClassifier()
modelo_knn = KNeighborsClassifier()

# Treinamento dos modelos com os dados de treino
modelo_arvoredecisao.fit(x_treino, y_treino)
modelo_knn.fit(x_treino, y_treino)

# Teste dos modelos com os dados de teste
previsao_arvoredecisao = modelo_arvoredecisao.predict(x_test)
previsao_knn = modelo_knn.predict(x_test.to_numpy())

# Avaliação da performance dos modelos
# A métrica "accuracy_score" compara as previsões com os valores reais (Y_test)
print("Acurácia - Árvore de Decisão:", accuracy_score(y_test, previsao_arvoredecisao))
print("Acurácia - KNN:", accuracy_score(y_test, previsao_knn))

# Decisão: Como o modelo de Árvore de Decisão apresentou melhor desempenho,
# ele será o modelo escolhido para uso.


c:\Users\João Pedro\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(


0.8237666666666666
0.7392333333333333


In [6]:
# PASSO 5: Importação e tratamento de nova base de dados

# Carrega a nova base de dados do arquivo CSV
tabela_nova = pandas.read_csv("novos_clientes.csv")

# Exibe as primeiras linhas da nova base de dados
display(tabela_nova)

# Exibe informações sobre a nova base de dados para verificar os tipos de dados e valores nulos
display(tabela_nova.info())

# As colunas "profissao", "mix_credito" e "comportamento_pagamento" precisam ser convertidas para o formato numérico,
# pois estão no formato "object", o que não é aceito pelo modelo de machine learning.

# Aplica o LabelEncoder para transformar as colunas categóricas em numéricas
tabela_nova["profissao"] = codificador.fit_transform(tabela_nova["profissao"])
tabela_nova["mix_credito"] = codificador.fit_transform(tabela_nova["mix_credito"])
tabela_nova["comportamento_pagamento"] = codificador.fit_transform(tabela_nova["comportamento_pagamento"])

# Confirma as alterações na base de dados
display(tabela_nova)
display(tabela_nova.info())


mes  idade   profissao  salario_anual  num_contas  num_cartoes  \
0    1   31.0  empresario      19300.340         6.0          7.0   
1    4   32.0    advogado      12600.445         5.0          5.0   
2    2   48.0  empresario      20787.690         8.0          6.0   

   juros_emprestimo  num_emprestimos  dias_atraso  num_pagamentos_atrasados  \
0              17.0              5.0         52.0                      19.0   
1              10.0              3.0         25.0                      18.0   
2              14.0              7.0         24.0                      14.0   

   ...  taxa_uso_credito idade_historico_credito  investimento_mensal  \
0  ...         29.934186                   218.0             44.50951   
1  ...         28.819407                    12.0              0.00000   
2  ...         34.235853                   215.0              0.00000   

       comportamento_pagamento  saldo_final_mes  emprestimo_carro  \
0  baixo_gasto_pagamento_baixo       312.487689                 1   
1  baixo_gasto_pagamento_medio       300.994163                 0   
2   baixo_gasto_pagamento_alto       345.081577                 0   

  emprestimo_casa  emprestimo_pessoal  emprestimo_credito  \
0               1                   0                   0   
1               0                   0                   0   
2               1                   0                   1   

   emprestimo_estudantil  
0                      0  
1                      1  
2                      0  

[3 rows x 23 columns]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 23 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   mes                       3 non-null      int64  
 1   idade                     3 non-null      float64
 2   profissao                 3 non-null      object 
 3   salario_anual             3 non-null      float64
 4   num_contas                3 non-null      float64
 5   num_cartoes               3 non-null      float64
 6   juros_emprestimo          3 non-null      float64
 7   num_emprestimos           3 non-null      float64
 8   dias_atraso               3 non-null      float64
 9   num_pagamentos_atrasados  3 non-null      float64
 10  num_verificacoes_credito  3 non-null      float64
 11  mix_credito               3 non-null      object 
 12  divida_total              3 non-null      float64
 13  taxa_uso_credito          3 non-null      float64
 14  idade_historic

None

mes  idade  profissao  salario_anual  num_contas  num_cartoes  \
0    1   31.0          1      19300.340         6.0          7.0   
1    4   32.0          0      12600.445         5.0          5.0   
2    2   48.0          1      20787.690         8.0          6.0   

   juros_emprestimo  num_emprestimos  dias_atraso  num_pagamentos_atrasados  \
0              17.0              5.0         52.0                      19.0   
1              10.0              3.0         25.0                      18.0   
2              14.0              7.0         24.0                      14.0   

   ...  taxa_uso_credito  idade_historico_credito  investimento_mensal  \
0  ...         29.934186                    218.0             44.50951   
1  ...         28.819407                     12.0              0.00000   
2  ...         34.235853                    215.0              0.00000   

   comportamento_pagamento  saldo_final_mes  emprestimo_carro  \
0                        1       312.487689                 1   
1                        2       300.994163                 0   
2                        0       345.081577                 0   

   emprestimo_casa  emprestimo_pessoal  emprestimo_credito  \
0                1                   0                   0   
1                0                   0                   0   
2                1                   0                   1   

   emprestimo_estudantil  
0                      0  
1                      1  
2                      0  

[3 rows x 23 columns]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 23 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   mes                       3 non-null      int64  
 1   idade                     3 non-null      float64
 2   profissao                 3 non-null      int32  
 3   salario_anual             3 non-null      float64
 4   num_contas                3 non-null      float64
 5   num_cartoes               3 non-null      float64
 6   juros_emprestimo          3 non-null      float64
 7   num_emprestimos           3 non-null      float64
 8   dias_atraso               3 non-null      float64
 9   num_pagamentos_atrasados  3 non-null      float64
 10  num_verificacoes_credito  3 non-null      float64
 11  mix_credito               3 non-null      int32  
 12  divida_total              3 non-null      float64
 13  taxa_uso_credito          3 non-null      float64
 14  idade_historic

None

In [7]:
# PASSO 6: Realizar novas previsões com a base de dados atualizada

# Utiliza o modelo de Árvore de Decisão treinado anteriormente para realizar previsões
# na nova base de dados.
previsoes = modelo_arvoredecisao.predict(tabela_nova)

# Exibe as previsões geradas pelo modelo
print(previsoes)


['Poor' 'Good' 'Standard']
